In [1]:
!pip install wget
import os
import torch
import pandas as pd
import numpy as np
import torch
from torch import nn


ssh_config = """
Host github.com
  IdentityFile ~/.ssh/github.pem
  User davipeag
  StrictHostKeyChecking no
"""

if os.name == 'nt':
  base_path = ""
  REPO_DIR = "."
  STORE_DIR ="." 
  print("Windows")
else:
  print("Unix-like")
  REPO_DIR = "/tmp/HeartRateRegression"
  from google.colab import drive
  drive.mount('/content/drive')
  GIT_PATH = "/content/drive/My\ Drive/deeplearning_project/github.pem"
  DATA_DIR = os.path.join(REPO_DIR, "repo")
  STORE_DIR ="/content/drive/My Drive/deeplearning_project/" 
  !mkdir ~/.ssh
  !cp -u {GIT_PATH} ~/.ssh/
  !chmod u=rw,g=,o= ~/.ssh/github.pem
  !echo "{ssh_config}" > ~/.ssh/config
  !chmod u=rw,g=,o= ~/.ssh/config
  ! (cd /tmp && git clone git@github.com:davipeag/HeartRateRegression.git)
  ! (cd {REPO_DIR} && git pull )
  import sys
  sys.path.append(REPO_DIR)

def git_pull():
  ! (cd {REPO_DIR} && git pull )

git_pull()

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=7587fd200a7632417e767c41d443ef36abdda3d1dc2afe2ac642d6a587b595e9
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
Unix-like
Mounted at /content/drive
Cloning into 'HeartRateRegression'...
remote: Enumerating objects: 163, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 548 (delta 99), reused 94 (delta 40), pack-reused 385
Receiving objects: 100% (548/548), 87.90 MiB | 35.96 MiB/s, done.
Resolving deltas: 100% (326/326), done.
Already up to date.
Already up to date.


In [2]:
import random
import torch
from torch import nn
import numpy as np
args = {
    'epoch_num': 250,     # Number of epochs.
    'lr': 1.0e-3,           # Learning rate.
    'weight_decay': 10e-4, # L2 penalty.
    'momentum': 0.9,      # Momentum.
    'num_workers': 0,     # Number of workers on data loader.
    'batch_size': 128,     # Mini-batch size. 128
    'batch_test': 248,     # size of test batch
    'window': 15,
    'initial_window':5,
    'clip_norm': 6.0,     # Upper limit on gradient L2 norm ###
}
if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

print(args['device'])

SEED = 1234
def reset_seeds():
  random.seed(SEED)
  np.random.seed(SEED)
  torch.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  torch.cuda.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

reset_seeds()

cuda


In [3]:
from data_utils import (PpgDaliaExtractor, FormatPPGDalia)

extractor = PpgDaliaExtractor(DATA_DIR)
ppg_dalia_formatter = FormatPPGDalia()
dfs_train = [ppg_dalia_formatter.transform(extractor.extract_subject(i)) for i in range(1,16)]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """


In [56]:
git_pull()

import importlib

import PPG
importlib.reload(PPG.AttentionDefaults)
importlib.reload(PPG)
importlib.reload(PPG.UtilitiesDataXY)
importlib.reload(PPG.Models)
importlib.reload(PPG.TrainerXY)

from PPG import FullTrainer

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From github.com:davipeag/HeartRateRegression
   9039be7..b25e066  master     -> origin/master
Updating 9039be7..b25e066
Fast-forward
 PPG/FullTrainer.py | 55 +++++++++++++++++++++++++++---------------------------
 1 file changed, 28 insertions(+), 27 deletions(-)


In [59]:
import random 
options = {
  'nfeatures': [4],
  'conv_filters': [64, 128, 256],
  'nconv_layers': [1, 2, 3, 4],
  'conv_dropout': [0, 0.1, 0.25,],
  'nenc_layers': [1, 2, 3, 4],
  'ndec_layers': [2, 3, 4],
  'nhead': [1, 2, 4],
  'feedforward_expansion': [1, 2, 4],
  'nlin_layers': [2,3,4],
  'lin_size': [16, 32, 64],
  'lin_dropout': [0, 0.25, 0.5],
  'lr': [0.001, 0.0001],
  'weight_decay': [0, 0.00001],
  'batch_size': [128, 256],
  'ts_sub': [0],
  'val_sub': [4]
 }

def choose(options):
  choice = dict()
  for k,v in options.items():
    choice[k] = random.choice(v)
  return choice


In [ ]:
while True:
  full_trainer = FullTrainer.AttentionFullTrainer(dfs_train, args["device"], 0, 1)
  out = full_trainer.train(**choose(options))
  print(out["args"], out["metric"])

[1/30]: loss_train: 11.529 loss_val 51.567 loss_ts 11.529
[3/30]: loss_train: 10.171 loss_val 42.507 loss_ts 10.171
best val epoch: 4
[4/30]: loss_train: 10.146 loss_val 21.554 loss_ts 10.146
[5/30]: loss_train: 10.351 loss_val 25.997 loss_ts 10.351
[6/30]: loss_train: 9.647 loss_val 26.581 loss_ts 9.647
[7/30]: loss_train: 9.526 loss_val 30.678 loss_ts 9.526
[8/30]: loss_train: 9.714 loss_val 26.867 loss_ts 9.714
[9/30]: loss_train: 9.795 loss_val 26.133 loss_ts 9.795
best val epoch: 10
[10/30]: loss_train: 9.977 loss_val 19.832 loss_ts 9.977
best val epoch: 11
[11/30]: loss_train: 10.066 loss_val 17.343 loss_ts 10.066
best val epoch: 12
[12/30]: loss_train: 9.696 loss_val 12.829 loss_ts 9.696
[13/30]: loss_train: 8.908 loss_val 27.167 loss_ts 8.908
[14/30]: loss_train: 9.318 loss_val 24.269 loss_ts 9.318
[15/30]: loss_train: 9.655 loss_val 19.205 loss_ts 9.655
[16/30]: loss_train: 9.259 loss_val 24.114 loss_ts 9.259
[17/30]: loss_train: 9.757 loss_val 20.354 loss_ts 9.757
[18/30]: lo

In [58]:
full_trainer = FullTrainer.AttentionFullTrainer(dfs_train, args["device"], 0, 1)

full_trainer.train()

best val epoch: 1
[1/30]: loss_train: 17.761 loss_val 37.989 loss_ts 17.761
[2/30]: loss_train: 13.198 loss_val 41.495 loss_ts 13.198
[3/30]: loss_train: 14.183 loss_val 44.654 loss_ts 14.183
[4/30]: loss_train: 13.810 loss_val 45.692 loss_ts 13.810
[5/30]: loss_train: 13.165 loss_val 48.072 loss_ts 13.165
[6/30]: loss_train: 14.161 loss_val 41.123 loss_ts 14.161
best val epoch: 7
[7/30]: loss_train: 13.948 loss_val 37.674 loss_ts 13.948
[8/30]: loss_train: 14.359 loss_val 42.283 loss_ts 14.359
[9/30]: loss_train: 13.799 loss_val 39.775 loss_ts 13.799
[10/30]: loss_train: 13.806 loss_val 40.962 loss_ts 13.806
best val epoch: 11
[11/30]: loss_train: 13.495 loss_val 37.275 loss_ts 13.495
[12/30]: loss_train: 12.646 loss_val 39.879 loss_ts 12.646
[13/30]: loss_train: 12.792 loss_val 40.446 loss_ts 12.792
[14/30]: loss_train: 13.287 loss_val 48.494 loss_ts 13.287
[15/30]: loss_train: 14.351 loss_val 45.145 loss_ts 14.351
[16/30]: loss_train: 14.426 loss_val 44.753 loss_ts 14.426
[17/30]: l

{'args': {'batch_size': 128,
  'conv_dropout': 0.5,
  'conv_filters': 64,
  'feedforward_expansion': 2,
  'lin_dropout': 0,
  'lin_size': 32,
  'lr': 0.001,
  'nconv_layers': 4,
  'ndec_layers': 2,
  'nenc_layers': 2,
  'nfeatures': 4,
  'nhead': 4,
  'nlin_layers': 2,
  'ts_sub': 0,
  'val_sub': 4,
  'weight_decay': 0.0001},
 'metric': 13.49531,
 'predictions': tensor([[83.5864],
         [99.5204],
         [99.5203],
         ...,
         [74.7939],
         [74.7939],
         [99.5206]])}

In [40]:
from PPG import UtilitiesDataXY 


transformers = PPG.AttentionDefaults.get_preprocessing_transformer()
make_loaders = UtilitiesDataXY.DataLoaderFactory(transformers, dfs_train).make_loaders

loader_tr, loader_val, loader_ts = make_loaders(ts_sub=0, val_sub=1)

In [41]:
from PPG.Models import SnippetConvolutionalTransformer

net = SnippetConvolutionalTransformer().to(args["device"])

# x,y = next(iter(loader_tr))

# p = net(x)

criterion = nn.MSELoss().to(args["device"])# nn.L1Loss().to(args["device"]) #nn.CrossEntropyLoss().to(args["device"])
optimizer = torch.optim.Adam(net.parameters(), lr=args["lr"],
                             weight_decay=args["weight_decay"])


In [42]:
from PPG.TrainerXY import (EpochTrainerXY, MetricsComputerXY, TrainHelperXY)
from preprocessing_utils import ZTransformer2

epoch_trainer = EpochTrainerXY(net, optimizer, criterion, args["device"])
ztransformer = ZTransformer2(['heart_rate', 'wrist-ACC-0', 'wrist-ACC-1', 'wrist-ACC-2',
              'wrist-BVP-0', 'wrist-EDA-0', 'wrist-TEMP-0', 'chest-ACC-0',
              'chest-ACC-1', 'chest-ACC-2', 'chest-Resp-0'])
metrics_comuter = MetricsComputerXY(ztransformer)

train_helper = TrainHelperXY(epoch_trainer, loader_tr, loader_val, loader_ts, metrics_comuter.mae)

In [43]:
train_helper.train(30)

[1/30]: loss_train: 17.742 loss_val 11.444 loss_ts 17.742
best val epoch: 2
[2/30]: loss_train: 12.161 loss_val 6.700 loss_ts 12.161
[3/30]: loss_train: 12.527 loss_val 7.174 loss_ts 12.527
best val epoch: 4
[4/30]: loss_train: 11.101 loss_val 6.087 loss_ts 11.101
best val epoch: 5
[5/30]: loss_train: 11.757 loss_val 5.934 loss_ts 11.757
[6/30]: loss_train: 9.287 loss_val 6.133 loss_ts 9.287
[7/30]: loss_train: 8.845 loss_val 6.866 loss_ts 8.845
[8/30]: loss_train: 9.448 loss_val 5.995 loss_ts 9.448
[9/30]: loss_train: 7.419 loss_val 7.544 loss_ts 7.419
[10/30]: loss_train: 8.696 loss_val 6.355 loss_ts 8.696
[11/30]: loss_train: 9.077 loss_val 6.228 loss_ts 9.077
[12/30]: loss_train: 8.272 loss_val 6.616 loss_ts 8.272
[13/30]: loss_train: 8.692 loss_val 6.357 loss_ts 8.692
[14/30]: loss_train: 8.125 loss_val 6.584 loss_ts 8.125
best val epoch: 15
[15/30]: loss_train: 7.320 loss_val 5.822 loss_ts 7.320
[16/30]: loss_train: 7.705 loss_val 5.927 loss_ts 7.705
[17/30]: loss_train: 7.387 lo

6.791491